# This is the new version of Skimpy from github

In [1]:
from skimpy.io.yaml import load_yaml_model, export_to_yaml

met_model = 'ecoli_shikki_regulation'
base = f'.'
path_to_kmodel = f'{base}/kinetic_model_scaffold.yaml'
output_model_path = f'{base}/output/kin_varma_curated_regulation.yaml'
kmodel = load_yaml_model(path_to_kmodel)

2024-03-04 18:21:59,692 - Unnamed - WARNING - Non integer stoichiometries found ['CYTBO3_4pp', 'LMPD_biomass_c_1_420', 'CYTBDpp'] change to integer for linear dependencies


In [2]:
from skimpy.io.regulation import load_enzyme_regulation
import pandas as pd
# Add regulation data to kinetic model
path_to_regulation_data = f'{base}/allosteric_regulations.csv'
df = pd.read_csv(path_to_regulation_data)
df_regulations_all = df[df['reaction_id'].isin(list(kmodel.reactions.keys()))]
df_regulations_all = df_regulations_all[df_regulations_all['regulator'].isin(list(kmodel.reactants.keys()))]

kmodel = load_enzyme_regulation(kmodel, df_regulations_all)

In [3]:
no_km = 0
no_ka = 0
no_ki = 0
k_list = []
for k,v in kmodel.parameters.items():
    if k.startswith("km_"):
        no_km += 1 
        k_list.append(k)
    elif ("activation" in k or "activator" in k):
        no_ka += 1
        k_list.append(k)
    elif ("inhibition" in k or "inhibitor" in k):
        no_ki += 1
        k_list.append(k)
print(f"The model has {len(kmodel.reactions)} reactions.")
print(f"The number of km in this model is {no_km}.")
print(f"The number of inhibition k in this model is {no_ki}.")
print(f"The number of activation k in this model is {no_ka}.")


The model has 123 reactions.
The number of km in this model is 358.
The number of inhibition k in this model is 12.
The number of activation k in this model is 10.


In [5]:
import pickle

file_name = "./output/parameter_names_k_with_regulation.pkl"
open_file = open(file_name, "wb")
pickle.dump(k_list, open_file)
open_file.close()

In [6]:
export_to_yaml(kmodel, output_model_path)

ValueError: dictionary update sequence element #0 has length 1; 2 is required